In [5]:
# langchain_example.py

import os
import json
import random
import dotenv
from typing import List, Dict, Any
from pydantic import BaseModel, Field

from langchain.agents import AgentExecutor, ZeroShotAgent
from langchain.tools import BaseTool
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from protocol import SocialGraph, find_potential_peers

dotenv.load_dotenv()

# Mockup data

AGENTS = {
    "Zubark": {"links": ["Quorvix", "Snurpik"], "skills": ["Python", "Data Analysis"]},
    "Quorvix": {"links": ["Varglith"], "skills": ["Java", "Machine Learning"]},
    "Phylobink": {"links": ["Varglith", "Yornath"], "skills": ["Python", "NLP"]},
    "Varglith": {"links": ["Blykatsu"], "skills": ["C++", "Computer Vision"]},
    "Yornath": {"links": ["Phylobink", "Zulfain"], "skills": ["JavaScript", "Web Development"]},
    'Blykatsu': {"links": ["Varglith", "Zulfain"], "skills": ["Ruby", "Go"]},
    "Zulfain": {"links": ["Blykatsu", "Snurpik"], "skills": ["Go", "Distributed Systems"]},
    "Snurpik": {"links": ["Zulfain", "Zubark"], "skills": ["Rust", "Blockchain"]},
}



# Define the Tool
from langchain.tools import BaseTool
from pydantic import Field


class FindPeerTool(BaseTool):
    name: str = "find_peer"
    description: str = "Find potential peers for a given task."
    social_graph: SocialGraph = Field(..., description="Social graph of agents.")

    def _run(self, tool_input: str) -> str:
        # Parse the tool_input to extract task_description, agent_id, and search_depth
        try:
            inputs = json.loads(tool_input)
            task_description = inputs.get("task_description")
            agent_id = inputs.get("agent_id")
            search_depth = inputs.get("search_depth", 2)
        except json.JSONDecodeError:
            return "Invalid input format. Expected JSON."

        if not task_description or not agent_id:
            return "Missing required parameters: 'task_description' and 'agent_id'."

        potential_peers = find_potential_peers(
            task_description, agent_id, self.social_graph, search_depth
        )
        return ', '.join(potential_peers) if potential_peers else "No potential peers found."

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

# Initialize the social graph
social_graph = SocialGraph(AGENTS)

# Instantiate the tool
find_peer_tool = FindPeerTool(social_graph=social_graph)

# Set random seed for reproducibility
random.seed(42)

# Define the agent creation function
def create_agent(agent_id: str):
    tools = [find_peer_tool]
    tool_names = ", ".join([tool.name for tool in tools])
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

    # Custom prompt with explicit parameter instructions
    prefix = f"""You are Agent {agent_id}. Your task is to find a peer for: {{input}}
Available tools:
{tool_descriptions}

When you decide to use a tool, provide the Action Input as a JSON string with the required parameters: 'task_description' and 'agent_id'.

Respond in the following format:

Thought: [Your thought process]
Action: [Tool name]
Action Input: [JSON string with 'task_description' and 'agent_id']
Observation: [Result of the action]
... (you can have multiple Thought/Action/Observation cycles)
Final Answer: [Your final answer]

Begin!
"""

    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=prefix,
        suffix="{agent_scratchpad}",
        input_variables=["input", "agent_scratchpad"],
    )

    llm_chain = LLMChain(
        llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
        prompt=prompt
    )
    agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True
    )
    return agent_executor

# Simulate Agent Interactions
def propagate_request(initial_agent: str, task: str, max_hops: int = 2):
    visited = set()
    current_agents = [initial_agent]
    connections = []

    for hop in range(max_hops):
        next_agents = []
        for agent_id in current_agents:
            if agent_id in visited:
                continue
            visited.add(agent_id)

            agent_executor = create_agent(agent_id)
            try:
                result = agent_executor.run(input=task)
            except Exception as e:
                print(f"Error running agent {agent_id}: {e}")
                continue

            # Extract chosen peers from the final answer
            # Handle cases where the final answer might include more text
            # For simplicity, assume the final answer is a comma-separated list
            chosen_peers = [peer.strip() for peer in result.split(',') if peer.strip()]

            for peer in chosen_peers:
                if peer in visited:
                    continue
                if peer in AGENTS.keys():
                    path = social_graph.get_path(agent_id, peer)
                    if path:
                        connections.append((agent_id, peer, path))
                        if peer not in visited:
                            next_agents.append(peer)

        current_agents = next_agents

    return connections

# Run the simulation
if __name__ == "__main__":
    task = "Develop a machine learning model for flat icon development"
    initial_agent = "Quorvix"
    connections = propagate_request(initial_agent, task)

    print(f"\nTask: {task}")
    print(f"Initial Agent: {initial_agent}")
    print("Connections formed:")
    if connections:
        for connection in connections:
            print(f"{connection[0]} -> {connection[1]} (Path: {' -> '.join(connection[2])})")
    else:
        print("No connections formed.")




> Entering new AgentExecutor chain...
Question: Find a peer for developing a machine learning model for flat icon development.

Thought: I need to find someone who has expertise in machine learning and experience in graphic design or icon development. This will ensure that the peer can effectively contribute to the task of developing a machine learning model specifically for flat icons.

Action: find_peer
Action Input: {"task_description":"Develop a machine learning model for flat icon development","agent_id":"Quorvix"}
Observation: No potential peers found.
Thought:It seems that there are currently no available peers for the specific task of developing a machine learning model for flat icon development. I may need to broaden the search criteria or consider alternative approaches to find someone with relevant skills.

Action: find_peer
Action Input: {"task_description":"Develop a machine learning model for graphic design","agent_id":"Quorvix"}
Observation: No potential peers found.
T

KeyboardInterrupt: 

In [4]:
# langchain_example.py

import os
import json
import random
import dotenv
from typing import List, Dict, Any
from pydantic import BaseModel, Field

from langchain.agents import AgentExecutor, ZeroShotAgent
from langchain.tools import BaseTool
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from protocol import SocialGraph, find_potential_peers

dotenv.load_dotenv()

# Mockup data

AGENTS = {
    "Zubark": {"links": ["Quorvix", "Snurpik"], "skills": ["Python", "Data Analysis"]},
    "Quorvix": {"links": ["Varglith"], "skills": ["Java", "Machine Learning"]},
    "Phylobink": {"links": ["Varglith", "Yornath"], "skills": ["Python", "NLP"]},
    "Varglith": {"links": ["Blykatsu"], "skills": ["C++", "Computer Vision"]},
    "Yornath": {"links": ["Phylobink", "Zulfain"], "skills": ["JavaScript", "Web Development"]},
    'Blykatsu': {"links": ["Varglith", "Zulfain"], "skills": ["Ruby", "Go"]},
    'Blykatsu': {"links": ["Varglith", "Zulfain"], "skills": ["Machine Learning", "Icon Design", "UX/UI"]},
    "Zulfain": {"links": ["Blykatsu", "Snurpik"], "skills": ["Go", "Distributed Systems"]},
    "Snurpik": {"links": ["Zulfain", "Zubark"], "skills": ["Rust", "Blockchain"]},
}



# Define the Tool
from langchain.tools import BaseTool
from pydantic import Field


class FindPeerTool(BaseTool):
    name: str = "find_peer"
    description: str = "Find potential peers for a given task."
    social_graph: SocialGraph = Field(..., description="Social graph of agents.")

    def _run(self, tool_input: str) -> str:
        # Parse the tool_input to extract task_description, agent_id, and search_depth
        try:
            inputs = json.loads(tool_input)
            task_description = inputs.get("task_description")
            agent_id = inputs.get("agent_id")
            search_depth = inputs.get("search_depth", 2)
        except json.JSONDecodeError:
            return "Invalid input format. Expected JSON."

        if not task_description or not agent_id:
            return "Missing required parameters: 'task_description' and 'agent_id'."

        potential_peers = find_potential_peers(
            task_description, agent_id, self.social_graph, search_depth
        )
        return ', '.join(potential_peers) if potential_peers else "No potential peers found."

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

# Initialize the social graph
social_graph = SocialGraph(AGENTS)

# Instantiate the tool
find_peer_tool = FindPeerTool(social_graph=social_graph)

# Set random seed for reproducibility
random.seed(42)

# Define the agent creation function
def create_agent(agent_id: str):
    tools = [find_peer_tool]
    tool_names = ", ".join([tool.name for tool in tools])
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

    # Custom prompt with explicit parameter instructions
    prefix = f"""You are Agent {agent_id}. Your task is to find a peer for: {{input}}
Available tools:
{tool_descriptions}

When you decide to use a tool, provide the Action Input as a JSON string with the required parameters: 'task_description' and 'agent_id'.

Respond in the following format:

Thought: [Your thought process]
Action: [Tool name]
Action Input: [JSON string with 'task_description' and 'agent_id']
Observation: [Result of the action]
... (you can have multiple Thought/Action/Observation cycles)
Final Answer: [Your final answer]

Begin!
"""

    prompt = ZeroShotAgent.create_prompt(
        tools,
        prefix=prefix,
        suffix="{agent_scratchpad}",
        input_variables=["input", "agent_scratchpad"],
    )

    llm_chain = LLMChain(
        llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
        prompt=prompt
    )
    agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True
    )
    return agent_executor

# Simulate Agent Interactions
def propagate_request(initial_agent: str, task: str, max_hops: int = 2):
    visited = set()
    current_agents = [initial_agent]
    connections = []

    for hop in range(max_hops):
        next_agents = []
        for agent_id in current_agents:
            if agent_id in visited:
                continue
            visited.add(agent_id)

            agent_executor = create_agent(agent_id)
            try:
                result = agent_executor.run(input=task)
            except Exception as e:
                print(f"Error running agent {agent_id}: {e}")
                continue

            # Extract chosen peers from the final answer
            # Handle cases where the final answer might include more text
            # For simplicity, assume the final answer is a comma-separated list
            chosen_peers = [peer.strip() for peer in result.split(',') if peer.strip()]

            for peer in chosen_peers:
                if peer in visited:
                    continue
                if peer in AGENTS.keys():
                    path = social_graph.get_path(agent_id, peer)
                    if path:
                        connections.append((agent_id, peer, path))
                        if peer not in visited:
                            next_agents.append(peer)

        current_agents = next_agents

    return connections

# Run the simulation
if __name__ == "__main__":
    task = "Develop a machine learning model for flat icon development"
    initial_agent = "Quorvix"
    connections = propagate_request(initial_agent, task)

    print(f"\nTask: {task}")
    print(f"Initial Agent: {initial_agent}")
    print("Connections formed:")
    if connections:
        for connection in connections:
            print(f"{connection[0]} -> {connection[1]} (Path: {' -> '.join(connection[2])})")
    else:
        print("No connections formed.")




> Entering new AgentExecutor chain...
Question: Find a peer for developing a machine learning model for flat icon development.

Thought: I need to find someone who has expertise in machine learning and experience in graphic design or icon development. This will ensure that the peer can contribute effectively to the task.

Action: find_peer
Action Input: {"task_description":"Develop a machine learning model for flat icon development","agent_id":"Quorvix"}

Observation: Blykatsu
Thought:I have found a potential peer named Blykatsu who may have the necessary skills for developing a machine learning model for flat icon development. 

Final Answer: The potential peer for developing a machine learning model for flat icon development is Blykatsu.

> Finished chain.

Task: Develop a machine learning model for flat icon development
Initial Agent: Quorvix
Connections formed:
No connections formed.


In [ ]:
AGENTS['Blykatsu']= {"links": ["Varglith", "Zulfain"], "skills": ["Machine Learning", "Icon Design", "UX/UI"]}


In [ ]:
# ... existing code ...

def example_social_graph_usage(agents_data):
    # Initialize the social graph with the provided agents data
    social_graph = SocialGraph(agents_data)

    # Example task
    task_description = "Develop a machine learning model for flat icon development"

    # Choose a random agent as the starting point
    initial_agent = random.choice(list(agents_data.keys()))

    print(f"Task: {task_description}")
    print(f"Initial Agent: {initial_agent}")

    # Find potential peers for the initial agent
    potential_peers = find_potential_peers(task_description, initial_agent, social_graph, search_depth=2)

    print("Potential peers found:")
    if potential_peers:
        for peer in potential_peers:
            path = social_graph.get_path(initial_agent, peer)
            print(f"  {peer} (Path: {' -> '.join(path)})")
    else:
        print("  No potential peers found.")

    # Demonstrate traversing the social graph
    print("\nTraversing the social graph:")
    for agent in social_graph.graph.nodes():
        neighbors = list(social_graph.graph.neighbors(agent))
        print(f"  {agent}'s neighbors: {', '.join(neighbors)}")

# Usage example
if __name__ == "__main__":
    example_social_graph_usage(AGENTS)